<a href="https://colab.research.google.com/github/mlabonne/Linear-Programming-Course/blob/main/3_Constraint_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Constraint Programming

> Chapter 3 of the [Linear Programming Course](https://github.com/mlabonne/Linear-Programming-Course)

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

Companion notebook to execute the code from the following article: 

In [ ]:
!python -m pip install --upgrade --user -q ortools

# Satisfiability

Find the number of soldiers in the enemy army, called $army$.

$$
  \left\{\begin{array}{@{}l@{}}
    army \equiv 0 \mod 13\\
    army \equiv 0 \mod 19\\
    army \equiv 0 \mod 37\\
    1 \leq army \leq 10\ 000
  \end{array}\right.\,
$$

If the following cell fails, click on `Runtime > Restart and run all`.

In [ ]:
from ortools.sat.python import cp_model

# Instantiate model and solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# 1. Variable
army = model.NewIntVar(1, 10000, 'army')

# 2. Constraints
# variable % mod = target → (target, variable, mod)
model.AddModuloEquality(0, army, 13)
model.AddModuloEquality(0, army, 19)
model.AddModuloEquality(0, army, 37)

# Find the variable that satisfies these constraints
status = solver.Solve(model)

# If a solution has been found, print results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('================= Solution =================')
    print(f'Solved in {solver.WallTime():.2f} milliseconds')
    print()
    print(f'🪖 Army = {solver.Value(army)}')
    print()
    print('Check solution:')
    print(f' - Constraint 1: {solver.Value(army)} % 13 = {solver.Value(army) % 13}')
    print(f' - Constraint 2: {solver.Value(army)} % 19 = {solver.Value(army) % 19}')
    print(f' - Constraint 3: {solver.Value(army)} % 37 = {solver.Value(army) % 37}')

else:
    print('The solver could not find a solution.')

================= Solution =================
Solved in 0.00 milliseconds

🪖 Army = 9139

Check solution:
 - Constraint 1: 9139 % 13 = 0
 - Constraint 2: 9139 % 19 = 0
 - Constraint 3: 9139 % 37 = 0


Print every solution with a callback with a new upper bound:

$$1 \leq army \leq 100\ 000$$

In [ ]:
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# 1. Variable
army = model.NewIntVar(1, 100000, 'army')

# 2. Constraints
model.AddModuloEquality(0, army, 13)
model.AddModuloEquality(0, army, 19)
model.AddModuloEquality(0, army, 37)


class PrintSolutions(cp_model.CpSolverSolutionCallback):
    """Callback to print every solution."""

    def __init__(self, variable):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variable = variable

    def on_solution_callback(self):
        print(self.Value(self.__variable))

# Solve with callback
solution_printer = PrintSolutions(army)
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, solution_printer)

9139
18278
27417
36556
45695
54834
63973
73112
82251
91390


# Optimization

The goal is to maximize the popularity of the rations without exceeding the capacity of 19.

|  | 🥖Bread | 🥩Meat | 🍺Beer |
| :--- | :---: | :---: | :---: |
| Size | 1 | 3 | 7 |
| Popularity    | 3 | 10 | 26 |

## 1. Constraint Programming solution:

In [ ]:
from ortools.sat.python import cp_model

# Instantiate model and solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# 1. Variables
capacity = 19
bread = model.NewIntVar(0, capacity, 'bread')
meat  = model.NewIntVar(0, capacity, 'meat')
beer  = model.NewIntVar(0, capacity, 'beer')

# 2. Constraints
model.Add(1 * bread
        + 3 * meat 
        + 7 * beer <= capacity)

# 3. Objective
model.Maximize(3  * bread
             + 10 * meat
             + 26 * beer)

# Solve problem
status = solver.Solve(model)

# If an optimal solution has been found, print results
if status == cp_model.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.WallTime():.2f} milliseconds')
    print()
    print(f'Optimal value = {3*solver.Value(bread)+10*solver.Value(meat)+26*solver.Value(beer)} popularity')
    print('Food:')
    print(f' - 🥖Bread = {solver.Value(bread)}')
    print(f' - 🥩Meat  = {solver.Value(meat)}')
    print(f' - 🍺Beer  = {solver.Value(beer)}')
    print()
    print(f'Constraint: 1*{solver.Value(bread)}🥖 + 3*{solver.Value(meat)}🥩 + 7*{solver.Value(beer)}🍺')
    print(f'            = {1*solver.Value(bread)+3*solver.Value(meat)+7*solver.Value(beer)} (<= {capacity})')
else:
    print('The solver could not find an optimal solution.')

================= Solution =================
Solved in 0.00 milliseconds

Optimal value = 68 popularity
Food:
 - 🥖Bread = 2
 - 🥩Meat  = 1
 - 🍺Beer  = 2

Constraint: 1*2🥖 + 3*1🥩 + 7*2🍺
            = 19 (<= 19)


## 2. Linear Programming solution:

In [ ]:
from ortools.linear_solver import pywraplp

# Instantiate solver with CBC backend
solver = pywraplp.Solver('',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# 1. Variables
capacity = 19
bread = solver.IntVar(0, capacity, 'bread')
meat  = solver.IntVar(0, capacity, 'meat')
beer  = solver.IntVar(0, capacity, 'beer')

# 2. Constraints
solver.Add(1 * bread
         + 3 * meat
         + 7 * beer <= capacity)

# 3. Objective
solver.Maximize(3  * bread
              + 10 * meat
              + 26 * beer)

# Solve problem
status = solver.Solve()

# If an optimal solution has been found, print results
if status == pywraplp.Solver.OPTIMAL:
  print('================= Solution =================')
  print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
  print()
  print(f'Optimal value = {solver.Objective().Value()} popularity')
  print('Food:')
  print(f' - 🥖Bread  = {bread.solution_value()}')
  print(f' - 🥩Meat = {meat.solution_value()}')
  print(f' - 🍺Beer = {beer.solution_value()}')
  print()
  print(f'Constraint: 1*{bread.solution_value()}🥖 + 3*{meat.solution_value()}🥩 + 7*{beer.solution_value()}🍺')
  print(f'            = {1*bread.solution_value()+3*meat.solution_value()+7*beer.solution_value()} (<= {capacity})')
else:
  print('The solver could not find an optimal solution.')

================= Solution =================
Solved in 97.00 milliseconds in 1 iterations

Optimal value = 68.0 popularity
Food:
 - 🥖Bread  = 2.0
 - 🥩Meat = 1.0
 - 🍺Beer = 2.0

Constraint: 1*2.0🥖 + 3*1.0🥩 + 7*2.0🍺
            = 19.0 (<= 19)


Count the number of possible solutions (it takes hours and hours with a capacity of 1000).

In [ ]:
class CountSolutions(cp_model.CpSolverSolutionCallback):
    """Count the number of solutions."""

    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1

    def solution_count(self):
        return self.__solution_count

solution_printer = CountSolutions()

# Instantiate model and solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# 1. Variables
capacity = 1000
bread = model.NewIntVar(0, capacity, 'bread')
meat  = model.NewIntVar(0, capacity, 'meat')
beer  = model.NewIntVar(0, capacity, 'beer')

# 2. Constraints
model.Add(1 * bread
        + 3 * meat 
        + 7 * beer <= capacity)

# Print results
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, solution_printer)
print(solution_printer.solution_count())